In [1]:
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import final_val

In [2]:
path = dir_path+'/data/'

spettriRAW = pd.DataFrame()
for i in ['0','1','2','3','4','5']:
    c = 'C' + i
    data = pd.read_csv(path + c + '.dat',header = None, skiprows=1,sep='\t')
    
    spettriRAW['λ'] = data.iloc[:, 0]
    spettriRAW[c] = data.iloc[:,1]

spettriRAW.tail()

,λ,C0,C1,C2,C3,C4,C5
55,205,-0.02454,-0.02172,-0.02031,-0.02233,-0.01931,-0.01872
56,204,-0.02265,-0.01981,-0.02289,-0.02221,-0.02085,-0.01986
57,203,-0.02256,-0.01913,-0.02492,-0.02548,-0.01868,-0.02221
58,202,-0.02118,-0.02115,-0.02197,-0.02395,-0.02055,-0.02537
59,201,-0.01932,-0.02146,-0.02278,-0.01982,-0.02226,-0.02139


In [3]:
N = 162 #numero residui BLG
l = 0.5 # lunghezza cammino ottico in cm
R = 8.314 # J/(mol * K)
T = 300 # K

In [4]:
# import concentrazioni* 1000000 #M

concentrazioni = np.array([0,1.529499455,3.093301471,4.668148484,6.130150005,7.735930045])

print('Concentrazione GuHCl [M]: ',concentrazioni)

Concentrazione GuHCl [M]:  [0.         1.52949946 3.09330147 4.66814848 6.13015    7.73593004]


In [5]:
import plotly.graph_objects as go

fig = go.Figure()
for conc,c in zip(['C0', 'C1', 'C2', 'C3', 'C4', 'C5'],concentrazioni):
    fig.add_trace(go.Scatter(
        x=spettriRAW['λ'],
        y=spettriRAW[conc],
        mode='lines',
        name=np.round(c,2)
    ))

fig.update_layout(
    xaxis_title='λ [nm]',
    yaxis_title='θ [deg]',
    # title='Dati raccolti',
    legend_title='C denaturante [M]',
    legend=dict(
        x=1,
        y=0,
        xanchor='right',
        yanchor='bottom',
        # font=dict(size=10),  # Make legend font smaller
    ),
    yaxis=dict(range=[-0.037, -0.013])
)

fig.write_image('./images/spettro_raccolto.png')
fig.write_html('./html/spettro_raccolto.html')
fig.show()

### APPAIO LE CODE

In [6]:
fig = go.Figure()

spettri = pd.DataFrame()

spettri['λ'] = spettriRAW['λ']

for i, conc in enumerate(['C0', 'C1', 'C2', 'C3', 'C4', 'C5']):
    λ = spettriRAW['λ']
    if i != 0:
        θ = spettriRAW[conc] * 100 / (concentrazioni[i] * N * l)
    else:
        θ = spettriRAW[conc] * 100 / (N * l)

    mean = θ[λ > 250].mean()
   #  θ = θ - mean
    spettri[conc] = θ  # aggiorna la colonna

    fig.add_trace(go.Scatter(
        x=λ,
        y=θ,
        mode='lines',
        name=np.round(concentrazioni[i],2)
    ))

fig.update_layout(
    yaxis=dict(range=[-0.08000, 0.04000]),
    xaxis_title='λ',
    yaxis_title='θ [deg·cm² dmol⁻¹]',
    legend_title='C denaturante [M]',
    legend=dict(
        x=1,
        y=0,
        xanchor='right',
        yanchor='bottom'
    )
)
fig.show()

In [7]:
λ = spettri['λ']

λmin = 220
λmax = 230

imin,imax = λ[λ == λmin].index[0], λ[λ == λmax].index[0]
# print(imin,imax)

Λ = np.arange(λmax,λmin-1,-1)

# Seleziona solo le colonne delle concentrazioni (senza le colonne λ)
spettro_ristretto = spettri.loc[imax:imin, ['C0','C1', 'C2', 'C3', 'C4', 'C5']]
spettro_ristretto = spettro_ristretto.T
spettro_ristretto.columns = Λ
spettro_ristretto.insert(0, 'Concentrazione', spettro_ristretto.index)
spettro_ristretto.reset_index(drop=True, inplace=True)
spettro_ristretto['Concentrazione'] = concentrazioni
display(spettro_ristretto)


,Concentrazione,230,229,228,227,226,225,224,223,222,221,220
0,0.000000,-0.029407,-0.029728,-0.029951,-0.030086,-0.030296,-0.030580,-0.030864,-0.031272,-0.031395,-0.032148,-0.032444
1,1.529499,-0.019453,-0.019493,-0.019574,-0.019792,-0.019735,-0.019840,-0.019969,-0.020090,-0.020260,-0.020583,-0.020777
2,3.093301,-0.009583,-0.009611,-0.009619,-0.009714,-0.009754,-0.009814,-0.009906,-0.009906,-0.009998,-0.010177,-0.010113
3,4.668148,-0.006278,-0.006239,-0.006329,-0.006363,-0.006339,-0.006405,-0.006408,-0.006437,-0.006416,-0.006442,-0.006427
4,6.130150,-0.004654,-0.004717,-0.004757,-0.004753,-0.004771,-0.004743,-0.004767,-0.004715,-0.004775,-0.004854,-0.004811
5,7.735930,-0.003733,-0.003742,-0.003725,-0.003763,-0.003792,-0.003726,-0.003752,-0.003717,-0.003784,-0.003734,-0.003787


In [8]:
# fig = go.Figure()
colors = px.colors.sample_colorscale(px.colors.sequential.Plasma, len(Λ))

# for idx,i in enumerate(range(λmin,λmax)):
#     fig.add_trace(go.Scatter(
#         x=spettro_ristretto['Concentrazione'],
#         y=spettro_ristretto[i],
#         mode='lines+markers',
#         name=str(i)+'nm',
#         line=dict(color=colors[idx]),
#         marker=dict(color=colors[idx])
#     ))

# fig.update_layout(
#     # yaxis_type='log',
#     #title='Spettri ristretti (220-230 nm)',
#     xaxis_title='Concentrazione [M]',
#     yaxis_title='θ',
#     legend_title='λ [nm]',
#     height=800,
#     width=600,
#     legend=dict(
#         x=1,
#         y=0,
#         xanchor='right',
#         yanchor='bottom'
#     )
# )
# # fig.show()

In [9]:
def sigmoid(x,m,Cmid, yN,yD):
    R = 8.314  # J/(mol * K)
    T = 300  # K
    expo = m * (Cmid-x) / (R*T)
    num = yN + yD * np.exp(-expo)
    den = 1 + np.exp(-expo)
    return num/den


# x = np.linspace(0,6,1000)
# plt.plot(x,sigmoid(x,-0.005,-0.0001,2,6))

# plt.show()

In [10]:
# fit con sigmoide

fit_results = []

for i in spettro_ristretto.columns[1:]:
    C = concentrazioni
    θ = spettro_ristretto[i].to_numpy()

    # guess
    yD = θ[-1]
    yN = θ[0] # NON è UN PARAMETRO
    m = 3898
    Cmid = np.mean(C)

    # fit
    popt, pcov = curve_fit(sigmoid, C, θ, p0=[m, Cmid, yN, yD],maxfev=1000,)
    perr = np.sqrt(np.diag(pcov))
    fit_results.append({
        'λ': i,
        'm': popt[0], 'm_err': perr[0],
        'Cmid': popt[1], 'Cmid_err': perr[1],
        'Cm covariance': pcov[1, 1],
        'yN': popt[2], 'yN_err': perr[2],
        'yD': popt[3], 'yD_err': perr[3],
    })

fit_df = pd.DataFrame(fit_results)
display(fit_df)

,λ,m,m_err,Cmid,Cmid_err,Cm covariance,yN,yN_err,yD,yD_err
0,230,2146.893567,320.616253,1.321189,0.306289,0.093813,-0.037613,0.003799,-0.003978,0.000493
1,229,2138.231504,304.706963,1.278895,0.300018,0.090011,-0.038371,0.003807,-0.003996,0.000470
2,228,2118.915642,333.826684,1.246279,0.341186,0.116408,-0.039001,0.004394,-0.004007,0.000522
3,227,2143.290193,326.338349,1.295317,0.316976,0.100474,-0.038696,0.004056,-0.004036,0.000511
4,226,2105.636744,299.463260,1.227384,0.313504,0.098284,-0.039663,0.004101,-0.004027,0.000476
5,225,2052.810236,311.955398,1.172075,0.355833,0.126617,-0.040781,0.004752,-0.003948,0.000517
6,224,2043.974888,293.729059,1.156103,0.341034,0.116304,-0.041346,0.004616,-0.003956,0.000493
7,223,2026.664821,301.942357,1.112666,0.365220,0.133386,-0.042401,0.005079,-0.003910,0.000516
8,222,2065.072593,281.242106,1.168080,0.317127,0.100569,-0.041862,0.004375,-0.003985,0.000473
9,221,2023.358155,275.201460,1.104996,0.335374,0.112475,-0.043702,0.004815,-0.003945,0.000485


In [11]:
fig = go.Figure()

C_plot = np.linspace(concentrazioni.min(), concentrazioni.max(), 200)

for idx, row in fit_df.iterrows():
    yN = row['yN']#spettro_ristretto.loc[0, row['λ']]
    yD = row['yD'] #spettro_ristretto.loc[5, row['λ']]
    # print(yN, yD)

    m = row['m']
    Cmid = row['Cmid']
    θ_fit = sigmoid(C_plot, m, Cmid, yN, yD)
    # Fit line
    fig.add_trace(go.Scatter(
        x=C_plot,
        y=θ_fit,
        mode='lines',
        line=dict(color=colors[idx]),
        name=f"Fit {int(row['λ'])} nm"
    ))
    # Experimental points
    fig.add_trace(go.Scatter(
        x=spettro_ristretto['Concentrazione'],
        y=spettro_ristretto[row['λ']],
        mode='markers',
        marker=dict(color=colors[idx]),
        name=f"Exp {int(row['λ'])} nm",
        showlegend=False
    ))

fig.update_layout(
    xaxis_title='Concentrazione [μM]',
    yaxis_title='θ (fit)',
    legend_title='λ [nm]',
    height=600,
    width=800,
    legend=dict(
        x=1,
        y=0,
        xanchor='right',
        yanchor='bottom'
    )
)

fig.show()

In [12]:
# Extract Cmid and its error
Cmid_values = fit_df['Cmid']
Cmid_errors = fit_df['Cmid_err']

# Compute mean and standard error of the mean
weights = 1/Cmid_errors**2
mean_Cmid = np.average(Cmid_values, weights=weights)
mean_Cmid_err = np.sqrt(np.sum(weights*((Cmid_values-mean_Cmid)**2))/np.sum(weights))



fig = go.Figure()

# markers
fig.add_trace(go.Scatter(
    x=fit_df['λ'],
    y=Cmid_values,
    error_y=dict(type='data', array=Cmid_errors, visible=True),
    mode='markers',
    name='Cmid'
))

mean_line_points = np.concatenate([[219],fit_df['λ'],[231]])
# mean dashed line
fig.add_trace(go.Scatter(
    x= mean_line_points,
    y=[mean_Cmid]*len(mean_line_points),
    mode='lines',
    line=dict(color='red', dash='dash'),
    name='Mean Cmid'
))

# bottom
fig.add_trace(go.Scatter(
    x=mean_line_points,
    y=[mean_Cmid - mean_Cmid_err]*len(mean_line_points),
    mode='lines',
    line=dict(color='red', width=0),
    showlegend=False
))
#top
fig.add_trace(go.Scatter(
    x=mean_line_points,
    y=[mean_Cmid + mean_Cmid_err]*len(mean_line_points),
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255,0,0,0.2)',
    line=dict(color='red', width=0),
    name='Mean ± SEM'
))

fig.update_layout(
    xaxis_title='λ [nm]',
    yaxis_title='Cmid',
    # title='Cmid vs λ with Mean and SEM',
    legend_title='Legend',
    yaxis=dict(range=[0, 2]),
    legend=dict(
        x=1,
        y=1,
        xanchor='right',
        yanchor='top'
    )
)

fig.show()

In [13]:
# Extract m and its error
m_values = fit_df['m']
m_errors = fit_df['m_err']

# Compute mean and standard error of the mean
weights_m = 1 / m_errors**2
mean_m = np.average(m_values, weights=weights_m)
mean_m_err = np.sqrt(np.sum(weights_m * ((m_values - mean_m) ** 2)) / np.sum(weights_m))

fig = go.Figure()

# markers with error bars
fig.add_trace(go.Scatter(
    x=fit_df['λ'],
    y=m_values,
    error_y=dict(type='data', array=m_errors, visible=True),
    mode='markers',
    name='m'
))

mean_line_points = np.concatenate([[λmin-1], fit_df['λ'], [λmax+1]])
# mean dashed line
fig.add_trace(go.Scatter(
    x=mean_line_points,
    y=[mean_m]*len(mean_line_points),
    mode='lines',
    line=dict(color='red', dash='dash'),
    name='Mean m'
))

# bottom
fig.add_trace(go.Scatter(
    x=mean_line_points,
    y=[mean_m - mean_m_err]*len(mean_line_points),
    mode='lines',
    line=dict(color='red', width=0),
    showlegend=False
))
# top
fig.add_trace(go.Scatter(
    x=mean_line_points,
    y=[mean_m + mean_m_err]*len(mean_line_points),
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255,0,0,0.2)',
    line=dict(color='red', width=0),
    name='Mean ± SEM'
))

fig.update_layout(
    xaxis_title='λ [nm]',
    yaxis_title='m',
    legend_title='Legend',
    yaxis=dict(range=[0, 5000]),
    legend=dict(
        x=1,
        y=1,
        xanchor='right',
        yanchor='top'
    )
)

fig.show()


In [14]:
# stampa medie

# print("Media pesata di Cmid:", mean_Cmid)
# print("Errore sulla media pesata di Cmid:", mean_Cmid_err)
# print("Media pesata di m:", mean_A)
# print("Errore sulla media pesata di m:", mean_A_err)

print('Cmid: ',final_val(mean_Cmid,mean_Cmid_err,4,udm='M'))
print('m: ', final_val(mean_m,mean_m_err, udm='LJ/mol2'))

ΔG = m * mean_Cmid
sigmaΔG = ΔG*np.sqrt((mean_m_err/mean_m)**2 + (mean_Cmid_err/mean_Cmid)**2)

print(ΔG,sigmaΔG)

Cmid:  1.21 ± 0.0701 M
m:  2084.23 ± 44.37 LJ/mol2
2538.0463510779928 156.64270688139723
